In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("data/train.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().any()

In [ ]:
df=df.rename(columns={'blue':'bluetooth','fc':'front_cam_mp','sc_h':'screen_ht','sc_w':'screen_wt'})

In [ ]:
df=df.rename(columns={'pc':'back_cam_mp'})

In [ ]:
df=df.drop(columns=['m_dep','mobile_wt','px_height','px_width'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['battery_power'].describe()

In [ ]:
df['bluetooth'].unique()

In [ ]:
df['clock_speed'].unique()

In [ ]:
df['dual_sim'].value_counts()

In [ ]:
df['front_cam_mp'].unique()

In [ ]:
df['four_g'].value_counts()

In [ ]:
df['int_memory'].describe()

In [ ]:
df['n_cores'].value_counts()

In [ ]:
df['back_cam_mp'].describe()

In [ ]:
df['ram'].describe()

In [ ]:
df['screen_ht'].unique()

In [ ]:
df['screen_wt'].unique()

In [ ]:
df['talk_time'].describe()

In [ ]:
df['three_g'].value_counts()

In [ ]:
df['touch_screen'].value_counts()

In [ ]:
df['four_g'].value_counts()

In [ ]:
df['wifi'].value_counts()

In [ ]:
df['price_range'].value_counts()

In [ ]:
df['clock_speed']=pd.cut(df['clock_speed'],bins=[0,1,2,3],labels=['low','mid','high'])



In [ ]:
df.head()

In [ ]:
level_map={'low':0,'mid':1,'high':2}
df['clock_speed']=df['clock_speed'].replace(level_map)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Bar Plot
fig = plt.figure(figsize = (6, 4))
title = fig.suptitle("No. of Cores Frequency", fontsize=14)
fig.subplots_adjust(top=0.85, wspace=0.3)

ax = fig.add_subplot(1,1, 1)
ax.set_xlabel("No.of Cores")
ax.set_ylabel("Frequency") 
w_q = df['n_cores'].value_counts()
w_q = (list(w_q.index), list(w_q.values))
ax.tick_params(axis='both', which='major', labelsize=8.5)
bar = ax.bar(w_q[0], w_q[1], color='steelblue', 
        edgecolor='black', linewidth=1)

In [ ]:
labels=['low','mid','high']
df['clock_speed'].value_counts().plot(kind='pie',autopct='%.2f')
plt.tight_layout()
plt.legend(labels)
plt.show()

In [ ]:

df['four_g'].value_counts().plot(kind='pie',autopct='%.2f')
plt.tight_layout()

plt.show()

In [ ]:
sns.set(style="ticks", color_codes=True)

In [ ]:
sns.barplot(x=df['price_range'],y=df['int_memory'])
plt.xticks(rotation=90)

In [ ]:

# Multi-bar Plot
cp = sns.countplot(x="price_range", hue="touch_screen", data=df, 
                   palette={0: "#FF9999", 1: "#FFE888"})

In [ ]:
# Box Plots
f, (ax) = plt.subplots(1, 1, figsize=(12, 4))
f.suptitle('No.of Cores - Internal Memory', fontsize=14)

sns.boxplot(x="n_cores", y="int_memory", data=df,  ax=ax)
ax.set_xlabel("No.of cores",size = 12,alpha=0.8)
ax.set_ylabel("internal memory",size = 12,alpha=0.8)

In [ ]:
df.boxplot(column='battery_power',by='price_range   ')
plt.show()

In [ ]:
df.show